### Split data into 10 folders

In [1]:
import os
import shutil

In [7]:
train_dir_cats = 'cats_or_dogs/train/cats/'
train_dir_dogs = 'cats_or_dogs/train/dogs/'
dest_dir ='Data/'

In [ ]:
i=1
new_path = os.path.join(dest_dir,str(i),'train/cats/')
os.makedirs(new_path)
count=0
for file in os.listdir(test_dir_cats):
    count+=1
    if count%50==0:
        i+=1
        new_path = os.path.join(dest_dir,str(i),'train/cats/')
        os.makedirs(new_path)
    shutil.copy(test_dir_cats + file, new_path + file)

### Import Libraries

In [2]:
import copy
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import models
from torchvision import datasets,transforms 

### Define transforms

In [3]:
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean,
                         std=std)
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean,
                         std=std)
])

### Create a list to hold all the models

In [4]:
models_ = list()

### Define train method

In [5]:
def train_model(model, trainloader, criterion, optimizer, num_epochs=25):

    total_step = len(trainloader)


    for epoch in range(num_epochs):

        for images,labels in trainloader:
    
                outputs = model(images)
                loss = criterion(outputs,labels)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
        if(epoch==9):
                print(epoch, loss.item())
                
    return model

### Define and train 10 models on their own data subsets

In [ ]:
for i in range(1,11):
    train_dir = os.path.join(dest_dir,str(i),'train')
    
    train_data= datasets.ImageFolder(root=train_dir,transform=train_transform)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=4,shuffle=True, num_workers=0)

    model = models.alexnet(pretrained=True)
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, 2)

    criterion   = nn.CrossEntropyLoss()
    optimizer   = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    model = train_model(model, train_loader,criterion, optimizer, num_epochs=10)
    
    models_.append(model)

### Aggregrate the model parameters

In [10]:
f_model = models.alexnet(pretrained=True)
num_ftrs = f_model.classifier[6].in_features
f_model.classifier[6] = nn.Linear(num_ftrs, 2)

In [11]:
beta = 0.5 #The interpolation parameter    

params = models_[0].named_parameters()
dict_params = dict(params)

for i in range(1,10):
    for name, param in models_[i].named_parameters():
        if name in dict_params:
            dict_params[name].data.copy_(beta*param.data + (1-beta)*dict_params[name].data)

f_model.load_state_dict(dict_params)

<All keys matched successfully>

In [12]:
f_model.eval() 
test_dir = 'cats_or_dogs/test'
test_data = datasets.ImageFolder(root=test_dir,transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=8, shuffle=True, num_workers=0)

with torch.no_grad():
    
    correct = 0
    total = 0
    
    for images, labels in test_loader:

        
        outputs = f_model(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {}%'\
          .format(100 * correct / total))

Accuracy of the model on the test images: 91.40316205533597%
